In [5]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package wordnet to C:\Users\Divyansh
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [10]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import gradient_descent_v2
import random

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(r'C:\Users\Divyansh Gupta\Desktop\College Assignments\Major 2\intents.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

41 documents
12 classes ['chatbot_name', 'food', 'goodbye', 'greeting', 'happy_emotion', 'joke', 'makers', 'noanswer', 'number', 'options', 'thanks', 'where']
56 unique lemmatized words ["'s", ',', 'a', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'could', 'created', 'creator', 'day', 'do', 'eat', 'for', 'from', 'funny', 'good', 'goodbye', 'happy', 'hello', 'help', 'helpful', 'hey', 'hi', 'hola', 'how', 'is', 'joke', 'later', 'made', 'me', 'name', 'next', 'nice', 'number', 'provide', 'say', 'see', 'something', 'tell', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'what', 'where', 'who', 'with', 'you', 'your']


In [7]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [8]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [11]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-11-9ffac2c3f218>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [12]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [13]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = gradient_descent_v2.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

D:\Anaconda\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [14]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
9/9 [==============================] - 3s 22ms/step - loss: 2.5125 - accuracy: 0.0732
Epoch 2/200
9/9 [==============================] - 0s 2ms/step - loss: 2.4093 - accuracy: 0.2683
Epoch 3/200
9/9 [==============================] - 0s 2ms/step - loss: 2.4004 - accuracy: 0.2439
Epoch 4/200
9/9 [==============================] - 0s 1ms/step - loss: 2.2732 - accuracy: 0.3415
Epoch 5/200
9/9 [==============================] - 0s 1ms/step - loss: 2.2813 - accuracy: 0.2683
Epoch 6/200
9/9 [==============================] - 0s 1ms/step - loss: 2.2028 - accuracy: 0.2683
Epoch 7/200
9/9 [==============================] - 0s 983us/step - loss: 2.1264 - accuracy: 0.2683
Epoch 8/200
9/9 [==============================] - 0s 1ms/step - loss: 2.0767 - accuracy: 0.2683
Epoch 9/200
9/9 [==============================] - 0s 1ms/step - loss: 2.0794 - accuracy: 0.3415
Epoch 10/200
9/9 [==============================] - 0s 1ms/step - loss: 2.0008 - accuracy: 0.4146
Epoch 11/200
9/9 [========

In [15]:
model.save(r'C:\Users\Divyansh Gupta\Desktop\College Assignments\Major 2\chatbot_model.h5', hist)